In [1]:
# !pip install statsmodels
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from sklearn.linear_model import LinearRegression

from statsmodels.regression.quantile_regression import QuantReg
import statsmodels.formula.api as smf
from scipy.stats import norm

In [2]:
url = 'https://raw.githubusercontent.com/mnuzen/156b-learning-support/master/data/us/covid/deaths.csv'
df = pd.read_csv(url, header=0)

In [3]:
cum_cases = df.iloc[:, 4:]
countyFIPS = df.iloc[:, 0].to_numpy()
cum_cases = cum_cases.to_numpy()
print(cum_cases.shape)

(3195, 130)


In [4]:
# predict the last two weeks
all_zeros = [0 for i in range(14)]
PREDS = []
reg = LinearRegression()
for i in range(len(cum_cases)):
    test = cum_cases[i, -14:]
    train = cum_cases[i, :-14]
    if train[-1] == 0: # no training
        PREDS.append(all_zeros)
    else:
        j = 0
        while train[j] == 0:
            j+=1
        j-=1
        y = train[j:]
        x = np.array(range(len(y))).reshape((len(y), 1))
        x_pred = np.array(range(len(y), len(y)+14)).reshape((14,1))
        reg.fit(x, y)
        pred = reg.predict(x_pred)
        PREDS.append(pred)

In [5]:
PREDS = np.array(PREDS)
print(PREDS.shape)
DAILY_PRED = np.zeros((3195,14))
DAILY_PRED[:, 0] = np.subtract(PREDS[:,0], cum_cases[:, -15])
for i in range(1, len(DAILY_PRED[0])):
    DAILY_PRED[:, i] = np.subtract(PREDS[:,i], PREDS[:, i-1])

(3195, 14)


In [6]:
FINAL_PRED = []
dates = np.loadtxt('dates.txt', dtype=np.str)
# assume normal distribution
for county in range(len(DAILY_PRED)):
    for date in range(len(DAILY_PRED[0])):
        mean = DAILY_PRED[county, date]
        std = max(1, mean)**(1/2)
        heading = dates[date] + '-' + str(countyFIPS[county])
        quantiles = np.linspace(norm.ppf(0.1, mean, std), norm.ppf(0.9, mean, std), 9)
        quantiles = quantiles.clip(0).tolist()
        quantiles.insert(0, heading)
        FINAL_PRED.append(quantiles)

In [7]:
FINAL_PRED = np.array(FINAL_PRED)
print(FINAL_PRED.shape)
print(FINAL_PRED[9960])

(44730, 10)
['2020-05-17-17203' '0.0' '0.0' '0.0' '0.0' '0.007936507936507908'
 '0.3283243993226579' '0.6487122907088081' '0.9691001820949583'
 '1.2894880734811083']


In [24]:
df = pd.DataFrame(data={"id": FINAL_PRED[:, 0], 
                        "10": FINAL_PRED[:, 1], 
                        "20": FINAL_PRED[:, 2], 
                        "30": FINAL_PRED[:, 3], 
                        "40": FINAL_PRED[:, 4], 
                        "50": FINAL_PRED[:, 5], 
                        "60": FINAL_PRED[:, 6], 
                        "70": FINAL_PRED[:, 7], 
                        "80": FINAL_PRED[:, 8], 
                        "90": FINAL_PRED[:, 9]})
df.to_csv("./test_linreg_daily.csv", sep=',',index=False)

In [26]:
# predict the last two weeks
all_zeros = [0 for i in range(14)]
PREDS = []
reg = LinearRegression()
for i in range(len(cum_cases)):
    if cum_cases[i, -1] == 0: # no training
        PREDS.append(all_zeros)
    else:
        j = 0
        y = cum_cases[i]
        while y[j] == 0:
            j+=1
        j-=1
        y = y[j:]
        x = np.array(range(len(y))).reshape((len(y), 1))
        x_pred = np.array(range(len(y), len(y)+14)).reshape((14,1))
        reg.fit(x, y)
        pred = reg.predict(x_pred)
        PREDS.append(pred)

In [27]:
PREDS = np.array(PREDS)
print(PREDS.shape)
DAILY_PRED = np.zeros((3195,14))
DAILY_PRED[:, 0] = np.subtract(PREDS[:,0], cum_cases[:, -1])
for i in range(1, len(DAILY_PRED[0])):
    DAILY_PRED[:, i] = np.subtract(PREDS[:,i], PREDS[:, i-1])

(3195, 14)


In [28]:
FINAL_PRED = []
dates = np.loadtxt('dates_final.txt', dtype=np.str)
# assume normal distribution
for county in range(len(DAILY_PRED)):
    for date in range(len(DAILY_PRED[0])):
        mean = DAILY_PRED[county, date]
        std = max(1, mean)**(1/2)
        heading = dates[date] + '-' + str(countyFIPS[county])
        quantiles = np.linspace(norm.ppf(0.1, mean, std), norm.ppf(0.9, mean, std), 9)
        quantiles = quantiles.clip(0).tolist()
        quantiles.insert(0, heading)
        FINAL_PRED.append(quantiles)

In [32]:
FINAL_PRED = np.array(FINAL_PRED)
print(FINAL_PRED.shape)
print(FINAL_PRED[43952])

(44730, 10)
['2020-05-31-55079' '2.012818209678896' '2.7168728160108686'
 '3.4209274223428414' '4.124982028674815' '4.829036635006787'
 '5.533091241338759' '6.237145847670733' '6.941200454002706'
 '7.645255060334678']


In [31]:
df = pd.DataFrame(data={"id": FINAL_PRED[:, 0], 
                        "10": FINAL_PRED[:, 1], 
                        "20": FINAL_PRED[:, 2], 
                        "30": FINAL_PRED[:, 3], 
                        "40": FINAL_PRED[:, 4], 
                        "50": FINAL_PRED[:, 5], 
                        "60": FINAL_PRED[:, 6], 
                        "70": FINAL_PRED[:, 7], 
                        "80": FINAL_PRED[:, 8], 
                        "90": FINAL_PRED[:, 9]})
df.to_csv("./submission_linreg_daily.csv", sep=',',index=False)

code demo also needed: Your code demo should be a Jupyter notebook which lays out exactly how to run your final model. This notebook should live in the parent directory of your git repo and be called ‘<team_name>_<team_number>_code_demo.ipynb’. The first cell should contain any data preprocessing you performed (smoothing, clustering, interpolation, etc). Subsequent cells should run your model, or models if your final submission was a blend. After all code cells are run the final output should be your final submission file for EvalAI. Note that we don’t want any of your actual model code in here, just calls to functions in other files which actually contain the model and brief descriptions of what each function call is doing. If some or all of your models are not written in Python the notebook should execute command line scripts which run your model. At the end, please include a rough estimate of how much time your model takes to run from beginning to end and what platform you run your models on (in case there are cross-platform compatibility issues). If you have models which did not make it into your final submission but which you’d like us to look at you’re welcome to add those into this notebook as well, just make it clear which model produced the actual final submission. While compiling this code demo you should also take the time to clean up your github repo and document your code, as this will form a small part of your final grade independent from the final report (see syllabus).